In [1]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
sales = pd.read_csv("../../Dataset/FINAL_LinkedCleanSalesWeatherWithEncoding.csv",index_col="date",parse_dates=True)

In [3]:
sales.head()

,station_nbr,item_nbr,units,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,...,smoke,widespread_dust,sandstorm,squall,freezing,shallow,partial,patches,blowing,vicinity
date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,1,1,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,2,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,3,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,4,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,5,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
sales.groupby("item_nbr")["units"].sum().sort_values(ascending=False).head()

item_nbr
45    1005111
9      916615
5      846662
44     577193
16     226772
Name: units, dtype: int64

Item <b>45</b> will be used as the main tester.

In [5]:
def getRegStr(df):
    lst = df.columns.values
    regStr = lst[2] + " ~ " + lst[3]
    for column in lst[4:]:
        regStr+=" + " + column
    return regStr       

In [6]:
def calculateMSE(test, target, model):
    test = test.reset_index()
    target = target.reset_index() 

    target["units_p"] = pd.DataFrame(model.predict(test))

    target.date = pd.to_datetime(target.date)
    target.set_index("date",inplace=True)

    target.units_p.fillna(0,inplace=True)
    
    # Rounding units did not improve MSE
    # Best MSE achieved by discarding prediction fractions
    #target.units_p = target.units_p.apply(round)
    target.units_p = target.units_p.astype(np.int64)
    target.units_p = target.units_p.apply(lambda x: 0 if x < 0 else x)

    return mean_squared_error(target.units,target.units_p)

In [7]:
def runRegression_backward(df, test_df, target_df):
    results = pd.DataFrame(columns=["model","rsquared_adj","MSE"])
    
    for feature in df.columns[3:]:
        reg = ols(getRegStr(df.drop(feature,axis=1)),df).fit()
        MSE = calculateMSE(test_df,target_df,reg)
        results = results.append(pd.DataFrame({"model":reg,"rsquared_adj":reg.rsquared_adj,"MSE" : MSE}, index = [feature]))
        
    return results

In [3]:
def saveFile(model, filename):
    #pickle.dump(model, open(filename, 'wb'))
    with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)
    
def loadFile(filename):
    return pickle.load(open(filename, 'rb'))

In [10]:
%%time
items = sales.item_nbr.unique()
foldsModels = pd.DataFrame(columns=["item_nbr","model","rsquared_adj","MSE"])
foldsGenerator = KFold(n_splits=5,shuffle=True,random_state=0)

for item in items:       # UN-comment to run on all items WARNING: runtime duration (~ 3-5 hrs).
#for item in range(1):     # UN-comment to run on single item only.
#    item = 45             # UN-comment to run on single item only.
    
    itemSales = sales.loc[sales.item_nbr == item].copy()
    
    ########################################################
    # TEST if any Feature is causing the regression to fail.    
    regAnalysis = pd.DataFrame(columns=["feature","rsquared_adj"])
    
    for feature in itemSales.columns[3:]:
        reg = ols("units~"+feature,itemSales).fit()
        regAnalysis = regAnalysis.append(pd.DataFrame({"feature":feature,"rsquared_adj" : reg.rsquared_adj}, index = [item]))
    badFeatures = regAnalysis.loc[regAnalysis.rsquared_adj.isnull()]
        
    itemSales.drop(badFeatures.feature.values,axis=1,inplace=True)
    if len(badFeatures)>0:
        print("Item [",item,"] Pre-processing: Bad Features dropped:",badFeatures.feature.values,"\n")
        print("Item [",item,"] Pre-processing: Remaining Features length",len(itemSales.columns)-3,"\n")
        if (len(itemSales.columns)-3) == 0:
            continue       # filter out items with no features to analyze.
    else:
        print("Item [",item,"] Pre-processing: Bad Features dropped: NONE\n")
    ########################################################
   
    folds = list(foldsGenerator.split(itemSales))

    for fold in range(len(folds)):   # Un-comment to run all Folds.
    #for fold in range(1):           # Un-comment to run Fold1 only.
        
        train, test = folds[fold]

        train_itemSales = itemSales.iloc[train]
        
        test_itemSales = itemSales.iloc[test]
        target_itemSales = itemSales.iloc[test][["units"]]
        
        orgModel = ols(getRegStr(train_itemSales),train_itemSales).fit()

        orgMSE = calculateMSE(test_itemSales,target_itemSales,orgModel)
        
        baseReg = orgModel.rsquared_adj
        baseMSE = orgMSE
        
        opt_itemSales = train_itemSales.copy()

        print("#####","Item [",item,"] Fold",fold+1,"#####")
        for i in range(len(opt_itemSales.columns[3:])):
            
            regAnalysis = runRegression_backward(opt_itemSales, test_itemSales, target_itemSales)
            
            maxReg = regAnalysis.loc[[regAnalysis.rsquared_adj.idxmax()]]
            minMSE = regAnalysis.loc[[regAnalysis.MSE.idxmin()]]
            
            ##############################################################
            #    Select Criteria HERE by setting only one mode as TRUE
            #-------------------------------------------------------------
            select_by_Radj = False
            select_by_MSE = True
            
            if select_by_Radj:
                criteria = baseReg < maxReg.rsquared_adj[0]
            elif select_by_MSE:
                criteria =  baseMSE > minMSE.MSE[0]
            #####################################################

            # Comment out to reduce printout.
            print("-----------------------------------------------------------------")
            print("Round:", i+1)
                       
            if select_by_Radj:
                print("Current Base reg:",baseReg)
                print("         Max Reg:", maxReg.rsquared_adj[0],"["+maxReg.index[0]+"]")
            elif select_by_MSE:
                print("Current Base MSE:",baseMSE)
                print("         Min MSE:", minMSE.MSE[0],"["+minMSE.index[0]+"]")

            if criteria:
                
                if select_by_Radj: 
                    opt_itemSales.drop(maxReg.index[0],axis=1,inplace=True)
                elif select_by_MSE:
                    opt_itemSales.drop(minMSE.index[0],axis=1,inplace=True)
                    
                baseReg = maxReg.rsquared_adj[0]
                baseMSE = minMSE.MSE[0]
                # Comment out to reduce printout.
                print("Continue")
                
            else:
                
                # Comment out to reduce printout.
                print("Break")
                print("-----------------------------------------------------------------")
                print("Item [",item,"] Fold",fold+1,"Results:")
                
                if select_by_Radj:
                    print(" Orignal score:", orgModel.rsquared_adj)
                    print("Improved score:", maxReg.rsquared_adj[0])
                elif select_by_MSE:
                    print(" Orignal MSE:", orgMSE)
                    print("Improved MSE:", baseMSE)
                    
                print("=================================================================")
                break

        optModel = ols(getRegStr(opt_itemSales),train_itemSales).fit()
        
        foldsModels = foldsModels.append(pd.DataFrame({"item_nbr":item,"model": optModel,\
                                                       "rsquared_adj" : optModel.rsquared_adj,\
                                                       "MSE" : calculateMSE(test_itemSales, target_itemSales, optModel)},\
                                                      index = ["fold"+str(fold+1)]))

foldsModels.index.name = "Folds"
foldsModels["selection"] = "backward"
foldsModels = foldsModels[["item_nbr","model","selection","rsquared_adj","MSE"]]     #rearrange columns

print("\n ***** Analysis Completed *****\n")

Item [ 45 ] Pre-processing: Bad Features dropped: NONE

##### Item [ 45 ] Fold 1 #####
-----------------------------------------------------------------
Round: 1
Current Base MSE: 13478.5228634
         Min MSE: 13477.5841045 [fog]
Continue
-----------------------------------------------------------------
Round: 2
Current Base MSE: 13477.5841045
         Min MSE: 13477.1878062 [mist]
Continue
-----------------------------------------------------------------
Round: 3
Current Base MSE: 13477.1878062
         Min MSE: 13476.9542733 [widespread_dust]
Continue
-----------------------------------------------------------------
Round: 4
Current Base MSE: 13476.9542733
         Min MSE: 13476.5389222 [rain]
Continue
-----------------------------------------------------------------
Round: 5
Current Base MSE: 13476.5389222
         Min MSE: 13476.5035384 [shallow]
Continue
-----------------------------------------------------------------
Round: 6
Current Base MSE: 13476.5035384
         Min MSE: 

In [114]:
foldsModels.head(n=10)

,,model,selection,rsquared_adj,MSE
Folds,item_nbr,,,,
fold1,1,<statsmodels.regression.linear_model.Regressio...,backward,0.045992,0.142079
fold2,1,<statsmodels.regression.linear_model.Regressio...,backward,0.047986,0.206042
fold3,1,<statsmodels.regression.linear_model.Regressio...,backward,0.042982,0.131500
fold4,1,<statsmodels.regression.linear_model.Regressio...,backward,0.045247,0.185135
fold5,1,<statsmodels.regression.linear_model.Regressio...,backward,0.048489,0.130139
fold1,2,<statsmodels.regression.linear_model.Regressio...,backward,0.061097,0.918617
fold2,2,<statsmodels.regression.linear_model.Regressio...,backward,0.062653,1.010343
fold3,2,<statsmodels.regression.linear_model.Regressio...,backward,0.061542,1.123332
fold4,2,<statsmodels.regression.linear_model.Regressio...,backward,0.066575,1.584536


In [ ]:
#Filter out the optimum model per item with the lowest MSE result.
foldsModels.set_index([foldsModels.index,"item_nbr"],inplace = True)
optFolds = foldsModels.groupby("item_nbr")["MSE"].idxmin().values
optFoldsModels = foldsModels.loc[optFolds]
optFoldsModels.index = optFoldsModels.index.droplevel(0)

In [71]:
optFoldsModels.to_pickle("BackwardRegressionMSE")

In [8]:
BackwardRegressionMSE = loadFile("BackwardRegressionMSE")

In [112]:
optFoldsModels.head()

,model,selection,rsquared_adj,MSE
item_nbr,,,,
1,<statsmodels.regression.linear_model.Regressio...,backward,0.048489,0.130139
2,<statsmodels.regression.linear_model.Regressio...,backward,0.061097,0.918617
3,<statsmodels.regression.linear_model.Regressio...,backward,0.089020,0.075939
4,<statsmodels.regression.linear_model.Regressio...,backward,0.007371,0.026946
5,<statsmodels.regression.linear_model.Regressio...,backward,0.358673,5964.703593


In [113]:
BackwardRegressionMSE.head()

,model,selection,rsquared_adj,MSE
item_nbr,,,,
1,<statsmodels.regression.linear_model.Regressio...,backward,0.048489,0.130139
2,<statsmodels.regression.linear_model.Regressio...,backward,0.061097,0.918617
3,<statsmodels.regression.linear_model.Regressio...,backward,0.089020,0.075939
4,<statsmodels.regression.linear_model.Regressio...,backward,0.007371,0.026946
5,<statsmodels.regression.linear_model.Regressio...,backward,0.358673,5964.703593


In [117]:
BackwardRegressionMSE.loc[[45]]

,model,selection,rsquared_adj,MSE
item_nbr,,,,
45,<statsmodels.regression.linear_model.Regressio...,backward,0.559505,13476.438759


In [107]:
testModel = BackwardRegressionMSE.loc[1,"model"]

In [108]:
testModel.rsquared_adj

0.04848896294339633

In [9]:
BackwardRegressionMSE

model selection  \
Folds item_nbr                                                                
fold5 1         <statsmodels.regression.linear_model.Regressio...  backward   
fold1 2         <statsmodels.regression.linear_model.Regressio...  backward   
fold2 3         <statsmodels.regression.linear_model.Regressio...  backward   
fold1 4         <statsmodels.regression.linear_model.Regressio...  backward   
      5         <statsmodels.regression.linear_model.Regressio...  backward   
      6         <statsmodels.regression.linear_model.Regressio...  backward   
fold5 7         <statsmodels.regression.linear_model.Regressio...  backward   
fold1 8         <statsmodels.regression.linear_model.Regressio...  backward   
fold5 9         <statsmodels.regression.linear_model.Regressio...  backward   
fold3 10        <statsmodels.regression.linear_model.Regressio...  backward   
fold2 11        <statsmodels.regression.linear_model.Regressio...  backward   
fold5 12        <statsmodels.regression.linear_model.Regressio...  backward   
fold2 13        <statsmodels.regression.linear_model.Regressio...  backward   
fold5 14        <statsmodels.regression.linear_model.Regressio...  backward   
fold3 15        <statsmodels.regression.linear_model.Regressio...  backward   
fold1 16        <statsmodels.regression.linear_model.Regressio...  backward   
fold5 17        <statsmodels.regression.linear_model.Regressio...  backward   
fold3 18        <statsmodels.regression.linear_model.Regressio...  backward   
fold1 19        <statsmodels.regression.linear_model.Regressio...  backward   
      20        <statsmodels.regression.linear_model.Regressio...  backward   
fold3 21        <statsmodels.regression.linear_model.Regressio...  backward   
fold5 22        <statsmodels.regression.linear_model.Regressio...  backward   
fold3 23        <statsmodels.regression.linear_model.Regressio...  backward   
fold1 24        <statsmodels.regression.linear_model.Regressio...  backward   
fold2 25        <statsmodels.regression.linear_model.Regressio...  backward   
fold4 26        <statsmodels.regression.linear_model.Regressio...  backward   
fold1 27        <statsmodels.regression.linear_model.Regressio...  backward   
fold4 28        <statsmodels.regression.linear_model.Regressio...  backward   
fold1 29        <statsmodels.regression.linear_model.Regressio...  backward   
      30        <statsmodels.regression.linear_model.Regressio...  backward   
...                                                           ...       ...   
fold2 82        <statsmodels.regression.linear_model.Regressio...  backward   
fold4 83        <statsmodels.regression.linear_model.Regressio...  backward   
fold3 84        <statsmodels.regression.linear_model.Regressio...  backward   
fold4 85        <statsmodels.regression.linear_model.Regressio...  backward   
fold2 86        <statsmodels.regression.linear_model.Regressio...  backward   
fold5 87        <statsmodels.regression.linear_model.Regressio...  backward   
fold1 88        <statsmodels.regression.linear_model.Regressio...  backward   
fold4 89        <statsmodels.regression.linear_model.Regressio...  backward   
      90        <statsmodels.regression.linear_model.Regressio...  backward   
fold5 91        <statsmodels.regression.linear_model.Regressio...  backward   
fold1 92        <statsmodels.regression.linear_model.Regressio...  backward   
fold2 93        <statsmodels.regression.linear_model.Regressio...  backward   
fold1 94        <statsmodels.regression.linear_model.Regressio...  backward   
fold3 95        <statsmodels.regression.linear_model.Regressio...  backward   
fold4 96        <statsmodels.regression.linear_model.Regressio...  backward   
fold2 97        <statsmodels.regression.linear_model.Regressio...  backward   
      98        <statsmodels.regression.linear_model.Regressio...  backward   
fold1 99        <statsmodels.regression.linear_model.Regressio...  backward   
fold5 100       <statsmodels.re